In [1]:

from WindPy import *
w.start()
w.isconnected()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [2]:
import pandas as pd
storage=pd.Series([])
storage


Series([], dtype: float64)

自建一个代码库，可以涵盖需要的股票信息

In [3]:
import datetime

class stock:
    def __init__(self,code,date):
        stock.code=code
        stock.name=w.wss(code, "sec_name").Data[0][0]
        stock.transdate=date
        stock.price=w.wsd(code,"close",date,).Data[0]
        
        
def historical(code,Days,freq=0):
    now = datetime.datetime.now()
    delta = datetime.timedelta(days=Days)
    old=now-delta
    storage=pd.DataFrame()
    freqdata="BarSize="+str(freq)
    if freq==0:
        a=now.strftime('%Y-%m-%d')
        b=old.strftime('%Y-%m-%d')
        z=w.wsd(code,"open,high,low,close,chg,amt",b, a, "")
    else:
        a=now.strftime('%Y-%m-%d %H:%M:%S')
        b=old.strftime('%Y-%m-%d %H:%M:%S')
        z=w.wsi(code, "open,high,low,close,chg,amt", a, b,freqdata)
    storage['Time']=z.Times[:]
    storage['Open']=z.Data[0]
    storage['High']=z.Data[1]
    storage['Low']=z.Data[2]
    storage['Close']=z.Data[3]
    storage['Change%']=z.Data[4]    
    storage['Amt']=z.Data[5]
    return storage
                

创建一个股票库，准备做多因子回归分析，基本想法是做一个df，以列的形式保存n天的数据，作为后续回归基础
可能存在的问题是数据量比较大（3759个A股公司）

In [4]:
#获取所有A股代码#
AllAStock =w.wset("SectorConstituent","date=20170502;sectorId=a001010100000000;field=wind_code");
if AllAStock.ErrorCode != 0:
    print("Get Data failed! exit!")
    exit()
#把股票代码转变为列表格式
stock_code=AllAStock.Data[0]
print(len(stock_code))

def database(codelist,Days):
    #codelist 传入一个列表形式的str，days为需要获取的时间长度#
    now = datetime.datetime.now()
    delta = datetime.timedelta(days=Days)
    old=now-delta
    a=now.strftime('%Y-%m-%d')
    b=old.strftime('%Y-%m-%d')
    data=pd.DataFrame()
    for i in codelist:
        data[i]=w.wsd(i,"close",b,a,"Fill=Previous").Data[0]
    return data

3206


写一个程序，输入两列（或三列）数字，返回R方，coef

In [5]:
import statsmodels.api as sm
def getregresult(a,b):
    #a is dependent, b is independent variable#
    b=sm.add_constant(b)
    model = sm.OLS(a, b).fit()
    intercept=model.params[0]
    slope=model.params[1]
    return model.params

In [6]:
a=database(stock_code[:2],30).iloc[:,0]
b=database(stock_code[:2],30).iloc[:,1]
print(getregresult(a,b))


const        7.294513
000002.SZ    0.296507
dtype: float64


C:\Users\zhang\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [ ]:
database(stock_code.ilo)

In [ ]:
#1. 简单的黄金交叉死亡交叉策略
# 移动平均线交叉进场条件：当FMA自下方与SMAj交叉时，进场做多；当FMA自上方与SMA交叉时，进场做空。